In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l', save_generated_data=False, additional_letter_attributes=False, reverse_inputs=False)
data.generate_data('TEST_train',
                   'TEST_test',
                   'TEST_validate')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [4]:
gen2 = data.generator('train', 16)
# test1 = next(gen1)
test2 = next(gen2)

In [7]:
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
# for el in test2[0][0]:
# #     print(el)
#     print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

10
(16, 23, 36)
(16, 140)


In [12]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 0
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')
print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.x_other_features_train[pos])
print(data.decode_y(data.y_train[pos]))

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
ašjenlautkajan
Agsnpn-
Agsnpn-
[1 1 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
[2, 5]


In [7]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 40
num_fake_epoch = 20



# conv_input_shape=(23, 36)
conv_input_shape=(23, 30)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

# x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x_conv)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [8]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch),
                              verbose=2)

Epoch 1/800
31s - loss: 0.2384 - actual_accuracy: 0.2243 - val_loss: 0.1819 - val_actual_accuracy: 0.4962
Epoch 2/800
38s - loss: 0.1726 - actual_accuracy: 0.4637 - val_loss: 0.1610 - val_actual_accuracy: 0.5637
Epoch 3/800
33s - loss: 0.1500 - actual_accuracy: 0.5509 - val_loss: 0.1400 - val_actual_accuracy: 0.6380
Epoch 4/800
34s - loss: 0.1322 - actual_accuracy: 0.6227 - val_loss: 0.1223 - val_actual_accuracy: 0.7017
Epoch 5/800
35s - loss: 0.1197 - actual_accuracy: 0.6776 - val_loss: 0.1194 - val_actual_accuracy: 0.7170
Epoch 6/800
33s - loss: 0.1092 - actual_accuracy: 0.7219 - val_loss: 0.1030 - val_actual_accuracy: 0.7623
Epoch 7/800
36s - loss: 0.1005 - actual_accuracy: 0.7510 - val_loss: 0.1013 - val_actual_accuracy: 0.7722
Epoch 8/800
36s - loss: 0.0959 - actual_accuracy: 0.7643 - val_loss: 0.0967 - val_actual_accuracy: 0.7872
Epoch 9/800
36s - loss: 0.0873 - actual_accuracy: 0.7920 - val_loss: 0.0983 - val_actual_accuracy: 0.7906
Epoch 10/800
35s - loss: 0.0836 - actual_accur

38s - loss: 0.0182 - actual_accuracy: 0.9630 - val_loss: 0.1301 - val_actual_accuracy: 0.8436
Epoch 79/800
33s - loss: 0.0188 - actual_accuracy: 0.9609 - val_loss: 0.1173 - val_actual_accuracy: 0.8600
Epoch 80/800
36s - loss: 0.0179 - actual_accuracy: 0.9624 - val_loss: 0.1031 - val_actual_accuracy: 0.8646
Epoch 81/800
39s - loss: 0.0174 - actual_accuracy: 0.9638 - val_loss: 0.1020 - val_actual_accuracy: 0.8631
Epoch 82/800
38s - loss: 0.0172 - actual_accuracy: 0.9649 - val_loss: 0.1151 - val_actual_accuracy: 0.8512
Epoch 83/800
40s - loss: 0.0177 - actual_accuracy: 0.9638 - val_loss: 0.0999 - val_actual_accuracy: 0.8585
Epoch 84/800
39s - loss: 0.0176 - actual_accuracy: 0.9636 - val_loss: 0.0929 - val_actual_accuracy: 0.8704
Epoch 85/800
35s - loss: 0.0172 - actual_accuracy: 0.9646 - val_loss: 0.1033 - val_actual_accuracy: 0.8631
Epoch 86/800
38s - loss: 0.0161 - actual_accuracy: 0.9667 - val_loss: 0.1053 - val_actual_accuracy: 0.8612
Epoch 87/800
39s - loss: 0.0168 - actual_accuracy:

39s - loss: 0.0118 - actual_accuracy: 0.9755 - val_loss: 0.1363 - val_actual_accuracy: 0.8547
Epoch 156/800
37s - loss: 0.0114 - actual_accuracy: 0.9766 - val_loss: 0.1303 - val_actual_accuracy: 0.8551
Epoch 157/800
35s - loss: 0.0116 - actual_accuracy: 0.9756 - val_loss: 0.1070 - val_actual_accuracy: 0.8719
Epoch 158/800
36s - loss: 0.0110 - actual_accuracy: 0.9765 - val_loss: 0.1182 - val_actual_accuracy: 0.8696
Epoch 159/800
37s - loss: 0.0120 - actual_accuracy: 0.9762 - val_loss: 0.1381 - val_actual_accuracy: 0.8551
Epoch 160/800
37s - loss: 0.0117 - actual_accuracy: 0.9761 - val_loss: 0.1072 - val_actual_accuracy: 0.8700
Epoch 161/800
39s - loss: 0.0118 - actual_accuracy: 0.9764 - val_loss: 0.1214 - val_actual_accuracy: 0.8639
Epoch 162/800
38s - loss: 0.0120 - actual_accuracy: 0.9767 - val_loss: 0.1346 - val_actual_accuracy: 0.8585
Epoch 163/800
38s - loss: 0.0110 - actual_accuracy: 0.9776 - val_loss: 0.1286 - val_actual_accuracy: 0.8635
Epoch 164/800
37s - loss: 0.0120 - actual_

36s - loss: 0.0088 - actual_accuracy: 0.9818 - val_loss: 0.1347 - val_actual_accuracy: 0.8662
Epoch 232/800
36s - loss: 0.0091 - actual_accuracy: 0.9823 - val_loss: 0.1552 - val_actual_accuracy: 0.8586
Epoch 233/800
38s - loss: 0.0089 - actual_accuracy: 0.9822 - val_loss: 0.1409 - val_actual_accuracy: 0.8574
Epoch 234/800
38s - loss: 0.0097 - actual_accuracy: 0.9802 - val_loss: 0.1331 - val_actual_accuracy: 0.8635
Epoch 235/800
38s - loss: 0.0092 - actual_accuracy: 0.9803 - val_loss: 0.1289 - val_actual_accuracy: 0.8827
Epoch 236/800
36s - loss: 0.0095 - actual_accuracy: 0.9813 - val_loss: 0.1603 - val_actual_accuracy: 0.8589
Epoch 237/800
37s - loss: 0.0087 - actual_accuracy: 0.9828 - val_loss: 0.1329 - val_actual_accuracy: 0.8746
Epoch 238/800
35s - loss: 0.0091 - actual_accuracy: 0.9803 - val_loss: 0.1288 - val_actual_accuracy: 0.8696
Epoch 239/800
37s - loss: 0.0092 - actual_accuracy: 0.9804 - val_loss: 0.1300 - val_actual_accuracy: 0.8689
Epoch 240/800
36s - loss: 0.0093 - actual_

36s - loss: 0.0084 - actual_accuracy: 0.9831 - val_loss: 0.1633 - val_actual_accuracy: 0.8597
Epoch 308/800
36s - loss: 0.0077 - actual_accuracy: 0.9838 - val_loss: 0.1496 - val_actual_accuracy: 0.8685
Epoch 309/800
39s - loss: 0.0074 - actual_accuracy: 0.9841 - val_loss: 0.1397 - val_actual_accuracy: 0.8724
Epoch 310/800
36s - loss: 0.0082 - actual_accuracy: 0.9826 - val_loss: 0.1573 - val_actual_accuracy: 0.8669
Epoch 311/800
37s - loss: 0.0071 - actual_accuracy: 0.9848 - val_loss: 0.1469 - val_actual_accuracy: 0.8731
Epoch 312/800
37s - loss: 0.0078 - actual_accuracy: 0.9846 - val_loss: 0.1437 - val_actual_accuracy: 0.8692
Epoch 313/800
36s - loss: 0.0077 - actual_accuracy: 0.9845 - val_loss: 0.1628 - val_actual_accuracy: 0.8677
Epoch 314/800
36s - loss: 0.0080 - actual_accuracy: 0.9833 - val_loss: 0.1332 - val_actual_accuracy: 0.8746
Epoch 315/800
36s - loss: 0.0080 - actual_accuracy: 0.9829 - val_loss: 0.1608 - val_actual_accuracy: 0.8719
Epoch 316/800
34s - loss: 0.0080 - actual_

...

34s - loss: 0.0069 - actual_accuracy: 0.9854 - val_loss: 0.1603 - val_actual_accuracy: 0.8662
Epoch 460/800
35s - loss: 0.0062 - actual_accuracy: 0.9860 - val_loss: 0.1490 - val_actual_accuracy: 0.8827
Epoch 461/800
37s - loss: 0.0070 - actual_accuracy: 0.9849 - val_loss: 0.1664 - val_actual_accuracy: 0.8570
Epoch 462/800
36s - loss: 0.0065 - actual_accuracy: 0.9868 - val_loss: 0.1964 - val_actual_accuracy: 0.8620
Epoch 463/800
37s - loss: 0.0066 - actual_accuracy: 0.9860 - val_loss: 0.1669 - val_actual_accuracy: 0.8682
Epoch 464/800
37s - loss: 0.0065 - actual_accuracy: 0.9864 - val_loss: 0.1706 - val_actual_accuracy: 0.8635
Epoch 465/800
36s - loss: 0.0065 - actual_accuracy: 0.9862 - val_loss: 0.1538 - val_actual_accuracy: 0.8769
Epoch 466/800
38s - loss: 0.0060 - actual_accuracy: 0.9874 - val_loss: 0.1455 - val_actual_accuracy: 0.8842
Epoch 467/800
35s - loss: 0.0070 - actual_accuracy: 0.9856 - val_loss: 0.1582 - val_actual_accuracy: 0.8681
Epoch 468/800
35s - loss: 0.0064 - actual_

38s - loss: 0.0066 - actual_accuracy: 0.9864 - val_loss: 0.1568 - val_actual_accuracy: 0.8738
Epoch 536/800
38s - loss: 0.0060 - actual_accuracy: 0.9866 - val_loss: 0.1854 - val_actual_accuracy: 0.8669
Epoch 537/800
36s - loss: 0.0056 - actual_accuracy: 0.9887 - val_loss: 0.1908 - val_actual_accuracy: 0.8746
Epoch 538/800
36s - loss: 0.0062 - actual_accuracy: 0.9868 - val_loss: 0.1666 - val_actual_accuracy: 0.8627
Epoch 539/800
36s - loss: 0.0069 - actual_accuracy: 0.9855 - val_loss: 0.1851 - val_actual_accuracy: 0.8639
Epoch 540/800
38s - loss: 0.0063 - actual_accuracy: 0.9864 - val_loss: 0.1789 - val_actual_accuracy: 0.8674
Epoch 541/800
37s - loss: 0.0061 - actual_accuracy: 0.9862 - val_loss: 0.1767 - val_actual_accuracy: 0.8654
Epoch 542/800
39s - loss: 0.0060 - actual_accuracy: 0.9873 - val_loss: 0.1516 - val_actual_accuracy: 0.8861
Epoch 543/800
38s - loss: 0.0058 - actual_accuracy: 0.9882 - val_loss: 0.1616 - val_actual_accuracy: 0.8785
Epoch 544/800
36s - loss: 0.0062 - actual_

33s - loss: 0.0051 - actual_accuracy: 0.9882 - val_loss: 0.1844 - val_actual_accuracy: 0.8650
Epoch 612/800
35s - loss: 0.0054 - actual_accuracy: 0.9892 - val_loss: 0.1846 - val_actual_accuracy: 0.8785
Epoch 613/800
38s - loss: 0.0058 - actual_accuracy: 0.9879 - val_loss: 0.1821 - val_actual_accuracy: 0.8600
Epoch 614/800
36s - loss: 0.0057 - actual_accuracy: 0.9863 - val_loss: 0.1742 - val_actual_accuracy: 0.8704
Epoch 615/800
33s - loss: 0.0061 - actual_accuracy: 0.9866 - val_loss: 0.1843 - val_actual_accuracy: 0.8712
Epoch 616/800
35s - loss: 0.0059 - actual_accuracy: 0.9876 - val_loss: 0.1763 - val_actual_accuracy: 0.8765
Epoch 617/800
40s - loss: 0.0052 - actual_accuracy: 0.9886 - val_loss: 0.2230 - val_actual_accuracy: 0.8636
Epoch 618/800
35s - loss: 0.0062 - actual_accuracy: 0.9869 - val_loss: 0.1783 - val_actual_accuracy: 0.8666
Epoch 619/800
34s - loss: 0.0065 - actual_accuracy: 0.9870 - val_loss: 0.1635 - val_actual_accuracy: 0.8838
Epoch 620/800
37s - loss: 0.0060 - actual_

38s - loss: 0.0056 - actual_accuracy: 0.9882 - val_loss: 0.1580 - val_actual_accuracy: 0.8712
Epoch 688/800
35s - loss: 0.0063 - actual_accuracy: 0.9868 - val_loss: 0.1820 - val_actual_accuracy: 0.8623
Epoch 689/800
36s - loss: 0.0051 - actual_accuracy: 0.9889 - val_loss: 0.1901 - val_actual_accuracy: 0.8650
Epoch 690/800
37s - loss: 0.0058 - actual_accuracy: 0.9872 - val_loss: 0.1763 - val_actual_accuracy: 0.8719
Epoch 691/800
39s - loss: 0.0052 - actual_accuracy: 0.9886 - val_loss: 0.1607 - val_actual_accuracy: 0.8842
Epoch 692/800
38s - loss: 0.0058 - actual_accuracy: 0.9884 - val_loss: 0.1854 - val_actual_accuracy: 0.8635
Epoch 693/800
39s - loss: 0.0057 - actual_accuracy: 0.9881 - val_loss: 0.2108 - val_actual_accuracy: 0.8640
Epoch 694/800
40s - loss: 0.0054 - actual_accuracy: 0.9882 - val_loss: 0.2072 - val_actual_accuracy: 0.8574
Epoch 695/800
37s - loss: 0.0056 - actual_accuracy: 0.9870 - val_loss: 0.1749 - val_actual_accuracy: 0.8689
Epoch 696/800
39s - loss: 0.0051 - actual_

38s - loss: 0.0053 - actual_accuracy: 0.9888 - val_loss: 0.1619 - val_actual_accuracy: 0.8754
Epoch 764/800
37s - loss: 0.0050 - actual_accuracy: 0.9884 - val_loss: 0.1814 - val_actual_accuracy: 0.8781
Epoch 765/800
40s - loss: 0.0056 - actual_accuracy: 0.9876 - val_loss: 0.1724 - val_actual_accuracy: 0.8769
Epoch 766/800
39s - loss: 0.0047 - actual_accuracy: 0.9895 - val_loss: 0.1901 - val_actual_accuracy: 0.8834
Epoch 767/800
39s - loss: 0.0061 - actual_accuracy: 0.9869 - val_loss: 0.1626 - val_actual_accuracy: 0.8669
Epoch 768/800
39s - loss: 0.0055 - actual_accuracy: 0.9882 - val_loss: 0.1647 - val_actual_accuracy: 0.8785
Epoch 769/800
36s - loss: 0.0050 - actual_accuracy: 0.9891 - val_loss: 0.1971 - val_actual_accuracy: 0.8681
Epoch 770/800
38s - loss: 0.0052 - actual_accuracy: 0.9885 - val_loss: 0.1873 - val_actual_accuracy: 0.8738
Epoch 771/800
36s - loss: 0.0047 - actual_accuracy: 0.9893 - val_loss: 0.2221 - val_actual_accuracy: 0.8547
Epoch 772/800
37s - loss: 0.0053 - actual_

In [10]:
name = '40_epoch'
# model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [7]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()